## Lab_6. Differential RNA expression analysis

In this project we will study changes that happen in yeast cells before or during fermentation and see in-detail the changes in RNA expression during the fermentation process.


[Link on results files ](https://drive.google.com/drive/folders/1lloDZ8gyPw7AWWTsxtzn7eDc9H3VVZW7)

### 1. Input data

There are two replicates of RNA-seq data from yeast before and during
fermentation, and my goal is to find out if the yeast express different genes during fermentation than they do under normal growth.


**yeast reads:**

**SRR941816: fermentation 0 minutes replicate 1**
ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941816/SRR941816.fastq.gz (413 Mb)

**SRR941817: fermentation 0 minutes replicate 2**
ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941817/SRR941817.fastq.gz (455 Mb)

**SRR941818: fermentation 30 minutes replicate 1**ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941818/SRR941818.fastq.gz (79.3 Mb)

**SRR941819: fermentation 30 minutes replicate 2**
ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941819/SRR941819.fastq.gz

---



As a reference genome we will use Saccharomyces cerevisiae, in the genome database at NCBI., strain S288c and assembly R64. Download the reference genome in FASTA format and annotation in GFF format.

**reference genome file**
ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.fna.gz

**annotation file**
ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.gff.gz

### 2. Analysis Pipeline

**Analysis flow chart**

**Inputs** - *fastq + reference genome + annotation*

⇓

**Pre-processing (indexing)**- *HISAT2*

⇓

**Alignement** - *HISAT2*

⇓

**Abundance estimation** - *featureCounts*

⇓

**Expression analysis and Visualization** - *DESeq2, gplots*




####**a) Aligning with HISAT2**

build genome index:

```bash
gunzip GCF_000146045.2_R64_genomic.fna.gz
ln -s GCF_000146045.2_R64_genomic.fna reference
hisat2-build -p 4 reference genome_index
```
run hisat2 in single-end mode:

``` python
SAMPLES = ["SRR941816", "SRR941817", "SRR941818", "SRR941819"]
HISAT2_INDEX = "/home/anna/BioInf_practice/6/genome_index"
THREADS = 4

rule all:
    input:
        expand("aligned/{sample}.bam", sample=SAMPLES)

rule extract_fastq:
    input:
        "{sample}.fastq.gz"
    output:
        "unpacked/{sample}.fastq"
    shell:
        "gunzip -c {input} > {output}"

rule align_and_sort:
    input:
        unpacked_fastq="unpacked/{sample}.fastq"
    output:
        bam="aligned/{sample}.bam"
    params:
        index=HISAT2_INDEX
    threads: THREADS
    shell:
        """
        hisat2 -p {threads} -x {params.index} -U {input.unpacked_fastq} | \
        samtools sort -@ {threads} -o {output.bam}
        """
snakemake --cores 4
```
####**b) Quantifying with featureCounts**

featureCounts can not work with GFF files. I converted the GFF file to GTF format with gffread.

```bash
mamba install gffread
ln -s GCF_000146045.2_R64_genomic.gff reference_gff
gffread GCF_000146045.2_R64_genomic.gff -T -o GCF_000146045.2_R64_genomic.gtf
```

To run the feature counts program I add 2 new rules to snakemake pipline:

```python
ANNOTATION_FILE = "/home/anna/BioInf_practice/6/reference_gtf"
rule all:
    input:
        expand("simple_counts/{sample}_simple_counts.txt", sample=SAMPLES)
        
rule featureCounts:
    input:
        bam="aligned/{sample}.bam"
    output:
        counts="counts/{sample}_counts.txt"
    params:
        annotation=ANNOTATION_FILE
    shell:
        """
        featureCounts -g gene_id -a {params.annotation} -o {output.counts} {input.bam}
        """

rule simplify_counts:
    input:
        counts="counts/{sample}_counts.txt"
    output:
        simple_counts="simple_counts/{sample}_simple_counts.txt"
    shell:
        """
        tail -n +2 {input.counts} | cut -f 1,7- > {output.simple_counts}
        """
```
And the pipline above also include the choice of necessary columns from the output files.


####**c) Find differentially expressed genes with Deseq2**

To calculate metrics and build a heatmap I used the following ran the following R scripts in R studio

```r
library(ggplot2)
library(pheatmap)
library(DESeq2)


SRR16 <- read.table(file("SRR941816_simple_counts.txt"), header=TRUE, row.names=1, sep="\t")
SRR17 <- read.table(file("SRR941817_simple_counts.txt"), header=TRUE, row.names=1, sep="\t")
SRR18 <- read.table(file("SRR941818_simple_counts.txt"), header=TRUE, row.names=1, sep="\t")
SRR19 <- read.table(file("SRR941819_simple_counts.txt"), header=TRUE, row.names=1, sep="\t")

combinedCountData <- cbind(SRR16, SRR17, SRR18, SRR19)
colnames(combinedCountData) <- c("SRR941816_control", "SRR941817_control", "SRR941818_exposure", "SRR941819_exposure")
conditions <- c("control", "control", "ferm30", "ferm30")
sample_info <- data.frame(Condition = conditions)
rownames(sample_info) <- c("SRR941816_control", "SRR941817_control", "SRR941818_exposure", "SRR941819_exposure")

sample_info$Condition <- factor(sample_info$Condition)

# DESeqDataSet object creation
dds <- DESeqDataSetFromMatrix(countData = combinedCountData,
                              colData = sample_info,
                              design = ~Condition)
dds$Condition <- factor(dds$Condition, levels = c('control', 'ferm30'))
dds <- DESeq(dds)

# Obtaining the results
res <- results(dds)

# Step 1. Results filtering
filtered_genes <- rownames(res)[res$baseMean > 0  & complete.cases(res) & res$padj < 0.05]

# Step 2: dds filtering
dds_filtered <- dds[filtered_genes,]
res_filtered <-  results(dds_filtered)
normalized_counts <- counts(dds_filtered, normalized=TRUE)

# Step 3: heatmap building
pheatmap(log1p(normalized_counts),
         clustering_distance_rows = "euclidean",
         clustering_distance_cols = "euclidean",  
         clustering_method = "complete",  
         show_rownames = FALSE,
         scale = 'row')
```



#### **3. Result Interpretation**

Then I took top-50 genes from the results sorted by sorted by adjusted p-values and saved them to a txt file to map them through GO (gene ontology) database.


```r
library(dplyr)
res_df <- as.data.frame(res_filtered)
res_sorted <- res_df %>%
  arrange(padj) %>%
  head(50)
top50_genes <- rownames(res_sorted)[1:50]
writeLines(top50_genes, "top50_genes.txt")
top50_dds <- dds_filtered[top50_genes, ]
res_top50 <- results(top50_dds)
normalized_counts_top50 <-counts(top50_dds, normalized=TRUE)
pheatmap(log1p(normalized_counts_top50),
         clustering_distance_rows = "euclidean",
         clustering_distance_cols = "euclidean",
         clustering_method = "complete",  
         show_rownames = TRUE)

```
After mapping selected genes to GO database results interpretation was started.